In [1]:
import numpy as np
from sdccode.modules import bayesianL
from sdccode.modules import cubicharmonics
import matplotlib.pyplot as plt
import copy

In [38]:
data=np.transpose(np.loadtxt('nlttk.out'))
sdata, grid = bayesianL.generatesorteddata(data, 500)
mN, SN, y_infer, sy_infer=bayesianL.bayesianpol(grid,sdata,7,70,0.00001,grid,True)

[0. 1. 0.] [0. 0. 1.] [1. 0. 0.]
parametri ottimali [ 8.55660904e-01 -2.43249610e+01  7.73733404e+01 -7.40904098e+02
  2.79165796e+01 -1.05581422e+03 -9.63653596e+01  2.83340128e+00
  2.16499691e+02  4.35943198e+01 -8.42982987e+00  7.31869581e+01
 -1.83876069e+00  9.71124335e-01 -1.42868502e+01 -1.27322313e+00]
numero di armoniche cubiche 8
valore a 0 inferito 0.6753076943224323 bias 0.8556609042120915 dato a kmin 0.6900529719009865
determinante matrice delle armoniche cubiche : 436169552821.62476


In [39]:
N=499
mN, SN, y_infer, sy_infer, spar, log_evidence_vP=bayesianL.bestfit(grid, sdata, N, grid, False, True)

parametri ottimali [ 5.41925303e-01 -6.18974435e+01 -4.27492757e+01  1.91490427e+03
  9.93931319e+02 -1.25088038e+04 -3.70804461e+02 -8.35487106e+02
 -1.21135329e+03 -6.56054301e+02  7.03857775e+02  4.74392254e+03
  6.98453269e+02  2.07173724e+02 -5.66868908e+02  5.93976653e+01
 -1.70401563e+03 -2.12935143e+03 -6.86764358e+02  1.56644198e+03
 -1.12232852e+03  3.21628250e+02  6.05076618e+02  2.50254391e+03
  3.73432724e+02 -4.69748775e+02 -1.10411959e+03 -2.12440056e+02
 -2.44755504e+03  1.38067197e+03 -5.51688037e+02  2.88763097e+03
 -1.48241951e+03  6.70271482e+02 -9.14831624e+02  4.50399326e+02
 -2.17650381e+02]
numero di armoniche cubiche 24
valore a 0 inferito 0.5017492135602546 bias 0.5419253026709899 dato a kmin 0.6900529719009865
determinante matrice delle armoniche cubiche : 8.261862720646216e-21
grado ottimale 28 grado massimo tentato 28


In [40]:
%matplotlib widget
plt.plot(log_evidence_vP, '.')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [41]:
%matplotlib widget
plt.plot(sdata[0], '.')
plt.plot(y_infer, '.')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [42]:
mr, sr, logevidence= bayesianL.bayesianmodelprediction(grid, sdata, N, grid, False, True)

model average prediction 0.36549811972920415 0.01271172911144589


In [43]:
def cubicarray(MM, pr=False):
    if pr: print('numero di elementi: ', len(MM))
    # In input bisogna fornire la lista dei monomi M. Ogni elemento della lista M e'
    # una lista [a,b,c] tale per cui M_i=x^{2a}y^{2b}z^{2c} e a+b+c=n, dove n e' il grado del monomio.
    # I monomi in input comprendono tutte le permutazioni.
    # Voglio ora applicare le permutazioni x,y,z in modo da simmetrizzare i monomi e ottenere le armoniche cubiche.
    lenM = len(MM)
    M=[]
    #Converto M a una lista di liste
    for i in range(len(MM)):
        M.append(list(MM[i]))
    Mt = copy.copy(M)
    # devo prima capire nella base dei monomi quali sono legati da permutazione. posso nominarli.
    nameC = []
    conta = 0
    contaci = 0
    if pr: print('numero cicli teornicamente necessari', int(len(M) ** 2 * 3 / 2))
    for i in range(lenM):
        rep = M[i]
        # rep (M[i]) e' il mio rappresentante nella lista,
        # voglio vedere quali altri elementi sono equivalenti per permutazione a rep
        nameC_i = []
        if not (rep in Mt): continue
        # se rep non sta nella lista temporanea Mt significa che e' equivalente a un elemento precedente: salto il ciclo
        nameC_i.append(np.array(rep))
        # in nameC_i inserisco gli elementi equivalenti per simmetria a rep
        for j in range(i + 1, lenM):
            # parto a cercare dall'elemento successivo nella lista, gli altri li ho gia' sicuramente controllati.
            Mjt = list(copy.copy(M[j]))
        
            if rep[0] in Mjt:
                Mjt.remove(rep[0])
                if rep[1] in Mjt:
                    Mjt.remove(rep[1])
                    if rep[2] in Mjt:
                        # rep e M[j] sono simmetrici per permutazione!
                        nameC_i.append(np.array(M[j]))
                        Mt.remove(M[j])
                        
                        # rimuovo da Mt l'elemento M[j] che e' simmetrico per permutazione a rep
                        conta += 1
        contaci += len(nameC_i)
        nameC.append(nameC_i)
        # aggiungo la lista degli elementi associati a M[i], nameC_i, alla lista degli elementi NON equivalenti, nameC!
    if contaci != len(M): raise ValueError
    if pr: print('numero cicli compiuti', conta)
    if pr: print('numero di elementi non equivalenti per simmetria cubica: ', len(nameC))
    # in output c'e' la lista degli elementi che formano una classe di equivalenza.
    return nameC

def datadicG(data, sigmadata, G):

    dicdata = {}
    
    dicsigma = {}
    
    for i in range(len(G)):
        dicdata[tuple(G[i])] = data[i]
        dicsigma[tuple(G[i])] = sigmadata[i]
        
    return dicdata, dicsigma

def datiplot(C, dic, dics):
    gplot=[]
    dataplot=[]
    datasigmaplot=[]
    for i in range(len(C)):
        gplot.append(np.linalg.norm(C[i][0]))
        dataav=0
        datasigma=0
        for j in range(len(C[i])):
            dataav+=dic[tuple(C[i][j])]/len(C[i])  
            datasigma+=(dics[tuple(C[i][j])]/len(C[i])  )**2
        dataplot.append(dataav)
        datasigmaplot.append(np.sqrt(datasigma))
    return np.array(gplot), np.array(dataplot), np.array(datasigmaplot)


In [44]:
C=cubicarray(list(grid), True)
dic, dics=datadicG(data[1], data[2],cubicharmonics.Gvecgenerateall(500)[1:])
gplot, dataplot, datasigmaplot=datiplot(C, dic, dics)
C=cubicarray(grid[:N], True)
dic, dics=datadicG(y_infer, np.sqrt(sy_infer), grid[:N])
gplotfit, dataplotfit, datasigmaplotfit=datiplot(C, dic, dics)
%matplotlib widget

plt.fill_between(gplotfit, dataplotfit+datasigmaplotfit, dataplotfit-datasigmaplotfit,  alpha=0.3, color='red')
plt.plot(gplotfit, dataplotfit, color='red')
plt.errorbar(gplot, dataplot,datasigmaplot, fmt='.')

numero di elementi:  499
numero cicli teornicamente necessari 373501
numero cicli compiuti 381
numero di elementi non equivalenti per simmetria cubica:  118
numero di elementi:  499
numero cicli teornicamente necessari 373501
numero cicli compiuti 381
numero di elementi non equivalenti per simmetria cubica:  118


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<ErrorbarContainer object of 3 artists>